In [1]:
import pandas
import numpy
import matplotlib


In [6]:
dataframe_ratings = pandas.read_csv('ml-32m/ratings.csv')
dataframe_tags = pandas.read_csv('ml-32m/tags.csv')
dataframe_movies = pandas.read_csv('ml-32m/movies.csv')

In [7]:
dataframe_ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [8]:
dataframe_tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


In [9]:
dataframe_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
dataframe_ratings_con_genres = dataframe_ratings.merge(
    dataframe_movies[['movieId', 'genres']], 
    on='movieId', 
    how='left'
)


In [12]:
# Verificar si hay filas con valores NA
filas_con_na = dataframe_ratings_con_genres.isnull().any(axis=1).sum()
print(f"Total de filas con valores NA: {filas_con_na}")

# Ver valores NA por columna
print("\nValores NA por columna:")
print(dataframe_ratings_con_genres.isnull().sum())

# Mostrar algunas filas con NA si existen
if filas_con_na > 0:
    print("\nPrimeras filas con valores NA:")
    print(dataframe_ratings_con_genres[dataframe_ratings_con_genres.isnull().any(axis=1)].head())


Total de filas con valores NA: 0

Valores NA por columna:
userId       0
movieId      0
rating       0
timestamp    0
genres       0
dtype: int64


In [13]:
# Extraer todos los géneros únicos
todos_generos = set()
for generos_str in dataframe_ratings_con_genres['genres'].dropna():
    if generos_str != '(no genres listed)':
        todos_generos.update(generos_str.split('|'))

# Crear diccionario de clasificación (género -> entero)
generos_sorted = sorted(list(todos_generos))
diccionario_generos = {genero: idx for idx, genero in enumerate(generos_sorted)}

# Convertir géneros de strings a enteros
def convertir_generos_a_enteros(generos_str):
    if pandas.isna(generos_str) or generos_str == '(no genres listed)':
        return []
    return [diccionario_generos[g] for g in generos_str.split('|')]

dataframe_ratings_con_genres['genres_enteros'] = dataframe_ratings_con_genres['genres'].apply(convertir_generos_a_enteros)


In [14]:
dataframe_ratings_con_genres.head()

,userId,movieId,rating,timestamp,genres,genres_enteros
0,1,17,4.0,944249077,Drama|Romance,"[7, 14]"
1,1,25,1.0,944250228,Drama|Romance,"[7, 14]"
2,1,29,2.0,943230976,Adventure|Drama|Fantasy|Mystery|Sci-Fi,"[1, 7, 8, 13, 15]"
3,1,30,5.0,944249077,Crime|Drama,"[5, 7]"
4,1,32,5.0,943228858,Mystery|Sci-Fi|Thriller,"[13, 15, 16]"
